In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mph
import pprint
from jpype.types import JInt, JDouble, JArray, JString

from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / 'util/'))

from functions import *

config = load_config( "config/Cavity_Qubit5G.yaml" )

In [3]:
client = mph.start()
pymodel = client.create('cavity')

In [4]:
parameters = pymodel/'parameters'
(parameters/'Parameters 1').rename('Parameters')

for parameter in config['parameter']:
    value = config['parameter'][parameter]
    pymodel.parameter( parameter, value)

pymodel.description('cavity_width', 'Cavity Width')
pymodel.description('cavity_length', 'Cavity Length')
pymodel.description('cavity_height', 'Cavity Height')
pymodel.description('cavity_fillet', 'Rounding Radius for cavity') 

In [5]:
## Java like
# model = pymodel.java
# model.component().create("comp1", True)
# model.component("comp1").geom().create("geom1", 3) # 3 means 3D
# model.component("comp1").geom("geom1").lengthUnit("\u00b5m")
# model.component("comp1").geom("geom1").create("wp1", "WorkPlane")
# model.component("comp1").geom("geom1").feature("wp1").set("unite", True)
# model.component("comp1").geom("geom1").feature("wp1").geom().create("r1", "Rectangle")
# model.component("comp1").geom("geom1").feature("wp1").geom().feature("r1").set("size", [2.4, 0.7])
# model.component("comp1").geom("geom1").feature("wp1").geom().feature("r1").set("pos", [-1.4, -0.35])
# model.component("comp1").geom("geom1").feature("wp1").geom().run("r1")

In [6]:


#
model = pymodel.java
comp1 = model.component().create("comp1", True)

geom1 = comp1.geom().create("geom1", 3) # 3 means 3D
geom1.lengthUnit("um")
geom1.run()

# Work plane 1
wp1 = geom1.create("wp1", "WorkPlane")
wp1.set("unite", True)
r1 = wp1.geom().create("r1", "Rectangle")
r1.set("size", ['cavity_width', 'cavity_length'])
r1.set("pos", ['-cavity_width/2', '0'])
wp1.geom().run("r1")

fil1 = wp1.geom().create("fil1", "Fillet")
fil1.selection("pointinsketch").set("r1", 1, 2, 3, 4)
fil1.set("radius", 'cavity_fillet')
wp1.geom().run("fil1")

wp1.label("cav-def")
geom1.run("wp1")

# Extrude 1
ext1 = geom1.create("ext1", "Extrude")
ext1.set("workplane", "wp1")
ext1.selection("input").set("wp1")
ext1.setIndex("distance", 'cavity_height', 0) # 0 means first row ?

geom1.run("ext1")

# Work plane 2
wp2 = geom1.create("wp2", "WorkPlane")
wp2.set("unite", True)
wp2.label("Ports-def")
wp2.set("quickplane", "xz")
c1 = wp2.geom().create("c1", "Circle")
c1.set("r", 'port_outer_radius')
c1.set("pos", ['0', '(cavity_height + port_distance)/2'] ) # Both x, y values must be int/float or string

mir1 = wp2.geom().create("mir1", "Mirror")
mir1.selection("input").set("c1")
mir1.set("pos", ['0', 'cavity_height/2'])
mir1.set("axis", [0., 1.])
mir1.set("keep", True)
wp2.geom().run("mir1")
geom1.run('wp2')

# Extrude 2

ext2 = geom1.create("ext2", "Extrude")
ext2.set("workplane", "wp2")
ext2.selection("input").set("wp2")
ext2.setIndex("distance", "port_height", 0)
ext2.label("Ports")

geom1.run("ext2")

# Work plane 3
wp3 = geom1.create("wp3", "WorkPlane")
wp3.set("unite", True)
wp3.set("quickplane", "xz")
wp3.set("quicky", "-port_height")
c2 = wp3.geom().create("c2", "Circle")
c2.set("pos",  ['0', '(cavity_height + port_distance)/2'] )
c2.set("r", 'port_inner_radius')
wp3.label("Pin-def")
geom1.run('wp3')

# Extrude 3
ext3 = geom1.create("ext3", "Extrude")
ext3.set("workplane", "wp3")
ext3.selection("input").set("wp3")
ext3.setIndex("distance", "-port_in_depth", 0)
ext3.label("Pin")
geom1.run("ext3")

# Work plane 4
wp4 = geom1.create("wp4", "WorkPlane")
wp4.set("unite", True)
wp4.set("quickplane", "xz")
wp4.set("quicky", "-port_height")
c3 = wp4.geom().create("c3", "Circle")
c3.set("pos",  ['0', '(cavity_height - port_distance)/2'] )
c3.set("r", 'port_inner_radius')
wp4.label("Pout-def")
geom1.run('wp4')

# Extrude 4
ext4 = geom1.create("ext4", "Extrude")
ext4.set("workplane", "wp4")
ext4.selection("input").set("wp4")
ext4.setIndex("distance", "-port_out_depth", 0)
ext4.label("Pout")
geom1.run("ext4")

# Work plane 5
wp5 = geom1.create("wp5", "WorkPlane")
wp5.set("unite", True)
wp5.set("quickz", 'cavity_height/2')
r2 = wp5.geom().create("r2", "Rectangle")
r2.set("size", ['wafer_space_width', 'wafer_space_length'])
r2.set("base", "center")
r2.set("pos", ['0', 'cavity_length/2'])
wp5.geom().run("r2")

fil2 = wp5.geom().create("fil2", "Fillet")
fil2.selection("pointinsketch").set("r2", 4, 3, 1, 2)
fil2.set("radius", 'wafer_space_fillet')
wp5.geom().run("fil2")

wp5.label("Chip space-def")
geom1.run("wp5")

# Extrude 5
ext5 = geom1.create("ext5", "Extrude")
ext5.set("workplane", "wp5")
ext5.selection("input").set("wp5")
ext5.setIndex("distance", '-wafer_space_height', 0)
ext5.label("Chip space")
geom1.run("ext5")

# Union 1
uni1 = geom1.create("uni1", "Union")
uni1.selection("input").set("ext1", "ext2", "ext5")
uni1.set("intbnd", False)
geom1.run("uni1")

# Work plane 6
wp6 = geom1.create("wp6", "WorkPlane")
wp6.set("unite", True)
wp6.set("quickz", "cavity_height/2 - wafer_space_height")
r3 = wp6.geom().create("r3", "Rectangle")
r3.set("base", "center")
r3.set("pos", ['0', 'cavity_length/2'])
r3.set("size", ['wafer_width', 'wafer_length'])
wp6.geom().run("r3")
wp6.label("Chip")
geom1.run("wp6")

# Extrude 6
ext6 = geom1.create("ext6", "Extrude")
ext6.set("workplane", "wp6")
ext6.selection("input").set("wp6")
ext6.setIndex("distance", 'wafer_height', 0)
geom1.run("ext6")

# Work plane 7
wp7 = geom1.create("wp7", "WorkPlane")
wp7.set("unite", True)
wp7.set("quickz", "cavity_height/2 - wafer_space_height + wafer_height")
r4 = wp7.geom().create("r4", "Rectangle")
r4.set("size", ['chip_width', 'chip_height'])
r4.set("base", "center")
r4.set("pos", ['0', 'cavity_length/2 + chip_height/2 + chip_gap/2'])
wp7.geom().run("r4")

fil3 = wp7.geom().create("fil3", "Fillet")
fil3.selection("pointinsketch").set("r4", 1, 2, 3, 4)
fil3.set("radius", 'chip_fillet')
wp7.geom().run("fil3")

r5 = wp7.geom().create("r5", "Rectangle")
r5.set("base", "center")
r5.set("pos", ['0', 'cavity_length/2 + jj_height/2 + finger_height/2'])
r5.set("size", ["jj_width", "finger_height"])
wp7.geom().run("r5")

# Union 2
uni2 = wp7.geom().create("uni2", "Union")
uni2.selection("input").set("fil3", "r5")
uni2.set("intbnd", False)
wp7.geom().run("uni2")

mir2 = wp7.geom().create("mir2", "Mirror")
mir2.selection("input").set("uni2")
mir2.set("pos", ['0', 'cavity_length/2'])
mir2.set("keep", True)
mir2.set("axis", [0., 1.])
wp7.geom().run("mir2")
wp7.label("Transmon pad")

r6 = wp7.geom().create("r6", "Rectangle")
r6.set("base", "center")
r6.set("size", ['jj_width','jj_height'])
r6.set("pos", ['0', 'cavity_length/2'])
wp7.geom().run("r6")

geom1.run("wp7")


# run geometry and set view after making everything
geom1.run()
comp1.view("view1").set("transparency", True)



<java object 'com.comsol.clientapi.impl.ViewClient'>

In [7]:
mat1 = comp1.material().create('mat1', 'Common')
mat1.label("Air")
mat1.set("family", "air")

mat1.propertyGroup("def").func().create("eta", "Piecewise")
mat1.propertyGroup("def").func().create("Cp", "Piecewise")
mat1.propertyGroup("def").func().create("rho", "Analytic")
mat1.propertyGroup("def").func().create("k", "Piecewise")
mat1.propertyGroup("def").func().create("cs", "Analytic")
mat1.propertyGroup("def").func().create("an1", "Analytic")
mat1.propertyGroup("def").func().create("an2", "Analytic")
mat1.propertyGroup().create("RefractiveIndex", "RefractiveIndex", "Refractive index")
mat1.propertyGroup().create("NonlinearModel", "NonlinearModel", "Nonlinear model")
mat1.propertyGroup().create("idealGas", "idealGas", "Ideal gas")
mat1.propertyGroup("idealGas").func().create("Cp", "Piecewise")
mat1.propertyGroup("def").func("eta").set("arg", "T")
mat1.propertyGroup("def").func("eta").set("pieces", [["200.0", "1600.0", "-8.38278E-7+8.35717342E-8*T^1-7.69429583E-11*T^2+4.6437266E-14*T^3-1.06585607E-17*T^4"]])
mat1.propertyGroup("def").func("eta").set("argunit", "K")
mat1.propertyGroup("def").func("eta").set("fununit", "Pa*s")
mat1.propertyGroup("def").func("Cp").set("arg", "T")
mat1.propertyGroup("def").func("Cp").set("pieces", [["200.0", "1600.0", "1047.63657-0.372589265*T^1+9.45304214E-4*T^2-6.02409443E-7*T^3+1.2858961E-10*T^4"]])
mat1.propertyGroup("def").func("Cp").set("argunit", "K")
mat1.propertyGroup("def").func("Cp").set("fununit", "J/(kg*K)")
mat1.propertyGroup("def").func("rho").set("expr", "pA*0.02897/R_const[K*mol/J]/T")
mat1.propertyGroup("def").func("rho").set("args", ["pA", "T"])
mat1.propertyGroup("def").func("rho").set("fununit", "kg/m^3")
mat1.propertyGroup("def").func("rho").set("argunit", ["Pa", "K"])
mat1.propertyGroup("def").func("rho").set("plotaxis", ["off", "on"])
mat1.propertyGroup("def").func("rho").set("plotfixedvalue", ["101325", "273.15"])
mat1.propertyGroup("def").func("rho").set("plotargs", [["pA", "101325", "101325"], ["T", "273.15", "293.15"]])
mat1.propertyGroup("def").func("k").set("arg", "T")
mat1.propertyGroup("def").func("k").set("pieces", [["200.0", "1600.0", "-0.00227583562+1.15480022E-4*T^1-7.90252856E-8*T^2+4.11702505E-11*T^3-7.43864331E-15*T^4"]])
mat1.propertyGroup("def").func("k").set("argunit", "K")
mat1.propertyGroup("def").func("k").set("fununit", "W/(m*K)")
mat1.propertyGroup("def").func("cs").set("expr", "sqrt(1.4*R_const[K*mol/J]/0.02897*T)")
mat1.propertyGroup("def").func("cs").set("args", ["T"])
mat1.propertyGroup("def").func("cs").set("fununit", "m/s")
mat1.propertyGroup("def").func("cs").set("argunit", ["K"])
mat1.propertyGroup("def").func("cs").set("plotfixedvalue", ["273.15"])
mat1.propertyGroup("def").func("cs").set("plotargs", [["T", "273.15", "373.15"]])
mat1.propertyGroup("def").func("an1").set("funcname", "alpha_p")
mat1.propertyGroup("def").func("an1").set("expr", "-1/rho(pA,T)*d(rho(pA,T),T)")
mat1.propertyGroup("def").func("an1").set("args", ["pA", "T"])
mat1.propertyGroup("def").func("an1").set("fununit", "1/K")
mat1.propertyGroup("def").func("an1").set("argunit", ["Pa", "K"])
mat1.propertyGroup("def").func("an1").set("plotaxis", ["off", "on"])
mat1.propertyGroup("def").func("an1").set("plotfixedvalue", ["101325", "273.15"])
mat1.propertyGroup("def").func("an1").set("plotargs", [["pA", "101325", "101325"], ["T", "273.15", "373.15"]])
mat1.propertyGroup("def").func("an2").set("funcname", "muB")
mat1.propertyGroup("def").func("an2").set("expr", "0.6*eta(T)")
mat1.propertyGroup("def").func("an2").set("args", ["T"])
mat1.propertyGroup("def").func("an2").set("fununit", "Pa*s")
mat1.propertyGroup("def").func("an2").set("argunit", ["K"])
mat1.propertyGroup("def").func("an2").set("plotfixedvalue", ["200"])
mat1.propertyGroup("def").func("an2").set("plotargs", [["T", "200", "1600"]])
mat1.propertyGroup("def").set("thermalexpansioncoefficient", "")
mat1.propertyGroup("def").set("molarmass", "")
mat1.propertyGroup("def").set("bulkviscosity", "")
mat1.propertyGroup("def").set("thermalexpansioncoefficient", ["alpha_p(pA,T)", "0", "0", "0", "alpha_p(pA,T)", "0", "0", "0", "alpha_p(pA,T)"])
mat1.propertyGroup("def").set("molarmass", "0.02897[kg/mol]")
mat1.propertyGroup("def").set("bulkviscosity", "muB(T)")
mat1.propertyGroup("def").set("relpermeability", ["1", "0", "0", "0", "1", "0", "0", "0", "1"])
mat1.propertyGroup("def").set("relpermittivity", ["1", "0", "0", "0", "1", "0", "0", "0", "1"])
mat1.propertyGroup("def").set("dynamicviscosity", "eta(T)")
mat1.propertyGroup("def").set("ratioofspecificheat", "1.4")
mat1.propertyGroup("def").set("electricconductivity", ["0[S/m]", "0", "0", "0", "0[S/m]", "0", "0", "0", "0[S/m]"])
mat1.propertyGroup("def").set("heatcapacity", "Cp(T)")
mat1.propertyGroup("def").set("density", "rho(pA,T)")
mat1.propertyGroup("def").set("thermalconductivity", ["k(T)", "0", "0", "0", "k(T)", "0", "0", "0", "k(T)"])
mat1.propertyGroup("def").set("soundspeed", "cs(T)")
mat1.propertyGroup("def").addInput("temperature")
mat1.propertyGroup("def").addInput("pressure")
mat1.propertyGroup("RefractiveIndex").set("n", ["1", "0", "0", "0", "1", "0", "0", "0", "1"])
mat1.propertyGroup("NonlinearModel").set("BA", "def.gamma-1")
mat1.propertyGroup("idealGas").func("Cp").label("Piecewise 2")
mat1.propertyGroup("idealGas").func("Cp").set("arg", "T")
mat1.propertyGroup("idealGas").func("Cp").set("pieces", [["200.0", "1600.0", "1047.63657-0.372589265*T^1+9.45304214E-4*T^2-6.02409443E-7*T^3+1.2858961E-10*T^4"]])
mat1.propertyGroup("idealGas").func("Cp").set("argunit", "K")
mat1.propertyGroup("idealGas").func("Cp").set("fununit", "J/(kg*K)")
mat1.propertyGroup("idealGas").set("Rs", "R_const/Mn")
mat1.propertyGroup("idealGas").set("heatcapacity", "Cp(T)")
mat1.propertyGroup("idealGas").set("ratioofspecificheat", "1.4")
mat1.propertyGroup("idealGas").set("molarmass", "0.02897[kg/mol]")
mat1.propertyGroup("idealGas").addInput("temperature")
mat1.propertyGroup("idealGas").addInput("pressure")
mat1.materialType("nonSolid")

<java object 'com.comsol.clientapi.impl.MaterialClient'>

In [8]:
mat2 = comp1.material().create('mat2', 'Common')
mat2.label("Silicon")
mat2.set("family", "custom")

mat2.propertyGroup().create("Enu", "Young's modulus and Poisson's ratio")
mat2.propertyGroup().create("RefractiveIndex", "Refractive index")
mat2.set("customspecular", [0.7843137254901961, 1, 1])
mat2.set("customdiffuse", [0.6666666666666666, 0.6666666666666666, 0.7058823529411765])
mat2.set("customambient", [0.6666666666666666, 0.6666666666666666, 0.7058823529411765])
mat2.set("noise", True)
mat2.set("fresnel", 0.7)
mat2.set("metallic", JInt(0))
mat2.set("pearl", JInt(0))
mat2.set("diffusewrap", JInt(0))
mat2.set("clearcoat", JInt(0))
mat2.set("reflectance", JInt(0))
mat2.propertyGroup("def").label("Basic")
mat2.propertyGroup("def").set("relpermeability", ["1", "0", "0", "0", "1", "0", "0", "0", "1"])
mat2.propertyGroup("def").set("electricconductivity", ["1e-12[S/m]", "0", "0", "0", "1e-12[S/m]", "0", "0", "0", "1e-12[S/m]"])
mat2.propertyGroup("def").set("thermalexpansioncoefficient", ["2.6e-6[1/K]", "0", "0", "0", "2.6e-6[1/K]", "0", "0", "0", "2.6e-6[1/K]"])
mat2.propertyGroup("def").set("heatcapacity", "700[J/(kg*K)]")
mat2.propertyGroup("def").set("relpermittivity", ["11.7", "0", "0", "0", "11.7", "0", "0", "0", "11.7"])
mat2.propertyGroup("def").set("density", "2329[kg/m^3]")
mat2.propertyGroup("def").set("thermalconductivity", ["130[W/(m*K)]", "0", "0", "0", "130[W/(m*K)]", "0", "0", "0", "130[W/(m*K)]"])
mat2.propertyGroup("Enu").label("Young's modulus and Poisson's ratio")
mat2.propertyGroup("Enu").set("E", "170[GPa]")
mat2.propertyGroup("Enu").set("nu", "0.28")
mat2.propertyGroup("RefractiveIndex").label("Refractive index")
mat2.propertyGroup("RefractiveIndex").set("n", ["3.48", "0", "0", "0", "3.48", "0", "0", "0", "3.48"])
mat2.set("family", "custom")
mat2.set("lighting", "cooktorrance")
mat2.set("fresnel", 0.7)
mat2.set("roughness", 0.5)
mat2.set("anisotropy", JInt(0))
mat2.set("flipanisotropy", False)
mat2.set("metallic", JInt(0))
mat2.set("pearl", JInt(0))
mat2.set("diffusewrap", JInt(0))
mat2.set("clearcoat", JInt(0))
mat2.set("reflectance", JInt(0))
mat2.set("ambient", "custom")
mat2.set("customambient", [0.6666666666666666, 0.6666666666666666, 0.7058823529411765])
mat2.set("diffuse", "custom")
mat2.set("customdiffuse", [0.6666666666666666, 0.6666666666666666, 0.7058823529411765])
mat2.set("specular", "custom")
mat2.set("customspecular", [0.7843137254901961, 1, 1])
mat2.set("noisecolor", "custom")
mat2.set("customnoisecolor", [0, 0, 0])
mat2.set("noisescale", JInt(0))
mat2.set("noise", "off")
mat2.set("noisefreq", JInt(1))
mat2.set("normalnoisebrush", "0")
mat2.set("normalnoisetype", "0")
mat2.set("alpha", JInt(1))
mat2.set("anisotropyaxis", [0, 0, 1])

mat1.selection().set(1, 3, 4)
mat2.selection().set(2)

<java object 'com.comsol.clientapi.impl.XDLocalSelectionClient'>

In [9]:
# C:\Program Files\COMSOL\COMSOL63\Multiphysics\data\comsol_basic_material_lib.mph
material_pymodel = client.load('C:\\Program Files\\COMSOL\\COMSOL63\\Multiphysics\\data\\comsol_basic_material_lib.mph')
material_model = material_pymodel.java
# materials = material_model.material('basic_material_lib_descr')
air = material_model.material('air')
print( air.propertyGroup() )
print( air.propertyGroup('def').func() )
print( air.propertyGroup('RefractiveIndex') )

pprint.pprint( dir(mat1) )
#mat1.copy(air)

Type: MaterialModelList
Tag: materialmodel
Child nodes: def, RefractiveIndex, NonlinearModel, idealGas
Type: Functions
Tag: func
Child nodes: eta, Cp, rho, k, cs, an1, an2
Type: Refractive Index
Tag: RefractiveIndex
Identifier: rfi
['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'active',
 'author',
 'comments',
 'dateCreated',
 'dateModified',
 'discardData',
 'docMarker',
 'equals',
 'feature',
 'getAPIEngine',
 'getAllowedPropertyValues',
 'getBoolean',
 'getBooleanArray',
 'getBooleanMatrix',
 'getClass',
 'getContainer',
 'getDisplayString',
 'getDouble',
 'getDoubleArray',
 'getDoubleMatrix',
 'getEntryKeyIndex',
 'getEntryKeys',
 'ge

In [10]:
mesh1 = comp1.mesh().create("mesh1")
emw  = comp1.physics().create("emw", "ElectromagneticWaves", "geom1")

lelement1 = emw.create("lelement1", "LumpedElement", 2)
lelement1.label("Zin")
lelement1.selection().set(13)
lelement1.set("PortType", "Coaxial")
lelement2 = emw.create("lelement2", "LumpedElement", 2)
lelement2.label("Zout")
lelement2.selection().set(10)
lelement2.set("PortType", "Coaxial")

lelement3 = emw.create("lelement3", "LumpedElement", 2)
lelement3.selection().set(43)
lelement3.set("LumpedElementType", "Inductor")
lelement3.set("Lelement", "chip_inductance")
lelement3.label("LJ")

<java object 'com.comsol.clientapi.physics.impl.PhysicsFeatureClient'>

In [11]:
std1 = model.study().create("std1")
eig = std1.create("eig", "Eigenfrequency")
eig.set("conrad", "1")
eig.set("linpsolnum", "auto")
eig.set("solnum", "auto")
eig.set("notsolnum", "auto")
eig.set("ngenAUX", "1")
eig.set("goalngenAUX", "1")
eig.setSolveFor("/physics/emw", True)

True

In [13]:
pymodel.save('mph/cavity_demo.mph')
client.remove(pymodel)